In [ ]:
# coefficents are encoded as unsigned integers: 255 => -1, 254 => -2, ...

import numpy as np
traces = np.load('../data/traces_hawk1024_10000.npy')
x = np.load('../data/x_hawk1024_10000.npy')
len(traces)

In [ ]:
repartition = {}
for i in range(10):
    repartition[i] = list(x).count(i)
for i in range(-6,0):
    repartition[i] = list(x).count(256+i)
assert sum(list(repartition.values())) == len(x)
repartition

In [ ]:
# to generate Figure 1

%matplotlib inline
import matplotlib.pyplot as plt
for i in range(20):
    plt.plot(traces[i][0:1750], linewidth=0.1, color='grey')
for i in range(6):
    plt.axvline(x=148*i+93, color='r')
for i in range(9):
    plt.axvline(x=84*i+837, color='b')
plt.show()

In [ ]:
# to generate all other figures

%matplotlib inline
import matplotlib.pyplot as plt
start = 123
end = 146
for i in range(20):
    # if x[i] % 2 == 0: # even coefficient
    # if x[i] > 128: # negative coefficient
    if x[i] == 0 or x[i] == 255 or x[i] == 1: # coefficient in [-1,1]
        plt.plot(traces[i][0:1750], linewidth=0.5, color='red')
    # elif x[i] == 0:
    #     plt.plot(traces[i][0:1750], linewidth=0.5, color='yellow')
    else:
        plt.plot(traces[i][0:1750], linewidth=0.5, color='blue')

plt.axis([start, end,-0.20,0.18])
plt.show()
print(x[:20])

In [ ]:
# to generate norm computation figure

%matplotlib inline
import matplotlib.pyplot as plt
start = 1524
end = 1547
for i in range(20):
    if x[i] > 128:
        plt.plot(traces[i][0:1750], linewidth=0.5, color='red')
    elif (x[i] & 4) != 0:
        plt.plot(traces[i][0:1750], linewidth=0.5, color='orange')
    else:
        plt.plot(traces[i][0:1750], linewidth=0.5, color='blue')
plt.axis([start, end,-0.25,0.2])
plt.show()
print(x[:20])

In [ ]:
# sign recovery during sign computation (zeros are splitted into the two categories)

means = []
for t in traces:
    means.append(np.mean(t[44:49]))
threshold = np.mean(means)
cat1 = []
cat2 = []
for i in range(len(traces)):
    if means[i] < threshold:
        cat1.append(x[i])
    else:
        cat2.append(x[i])
print(all(e > 128 or e == 0 for e in cat1))
print(all(e < 128 or e == 0 for e in cat2))
cat1.count(0), cat2.count(0)

In [ ]:
# sign recovery during sign application (zeros are splitted into the two categories)

means = []
for t in traces:
    means.append(np.mean(t[1510:1525]))
threshold = np.mean(means)
cat1 = []
cat2 = []
for i in range(len(traces)):
    if means[i] < threshold:
        cat1.append(x[i])
    else:
        cat2.append(x[i])

print(all(e > 128 or e == 0 for e in cat1))
print(all(e < 128 or e == 0 for e in cat2))
cat1.count(0), cat2.count(0)

In [ ]:
# sign recovery during norm computation (all zero are classified as positive)

means = []
for t in traces:
    means.append(np.mean(t[1529:1541]))
threshold = np.mean(means)
cat1 = []
cat2 = []
for i in range(len(traces)):
    if means[i] < threshold:
        cat1.append(x[i])
    else:
        cat2.append(x[i])

print(len(cat1), len(cat2))
print(all(e > 128 for e in cat1))
print(all(e < 128 for e in cat2))
cat1.count(0), cat2.count(0)

In [ ]:
# parity recovery during parity computation

means = []
for t in traces:
    means.append(np.mean(t[73:81]))
threshold = np.mean(means)
cat1 = []
cat2 = []
for i in range(100):
    if means[i] < threshold:
        cat1.append(x[i])
    else:
        cat2.append(x[i])
print(all(e % 2 == 1 for e in cat1))
print(all(e % 2 == 0 for e in cat2))

In [ ]:
# parity recovery during the choice between the two tables

means = []
for t in traces:
    means.append(np.mean(t[760:765]))
threshold = np.mean(means)
cat1 = []
cat2 = []
for i in range(len(traces)):
    if means[i] < threshold:
        cat1.append(x[i])
    else:
        cat2.append(x[i])
print(all(e % 2 == 1 for e in cat1))
print(all(e % 2 == 0 for e in cat2))

In [ ]:
# classify coefficients as either in [-1,1] or stricly greater (first incrementation of r)

means = []
index = 0
for t in traces:
    means.append(np.mean(t[128+148*index:140+148*index]))
threshold = np.mean(means)
cat1 = []
cat2 = []
for i in range(len(traces)):
    if means[i] < threshold:
        if (x[i] > 128):
            cat1.append(x[i] - 256)
        else:
            cat1.append(x[i])
    else:
        if (x[i] > 128):
            cat2.append(x[i] - 256)
        else:
            cat2.append(x[i])
print(all(abs(e) > 2*index + 1 for e in cat1))
print(all(abs(e) <= 2*index + 1 for e in cat2))

len(cat2)

In [ ]:
# set index = 0 to classify coefficients as either in [-1,1] or stricly greater (first incrementation of r)
# set index = 1 to classify coefficients as either in [-3,3] or stricly greater (second incrementation of r)
# set index = 2 to obtain some coefficients in [-5,5] (disregard the other classification)

from sklearn.cluster import KMeans

index = 1
means = []
for t in traces:
    means.append(np.mean(t[128+148*index:135+148*index]))

means = np.array(means)
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(means.reshape(-1,1))

predictions = kmeans.predict(means.reshape(-1,1))
cat = {0:[], 1:[]}
for i in range(10000):
    if x[i] < 128:
        cat[predictions[i]].append(x[i])
    else:
        cat[predictions[i]].append(x[i]-256)
print(len(cat[0]), len(cat[1]))
print(all(abs(e) > (2*index+1) for e in cat[0]) or all(abs(e) <= (2*index+1) for e in cat[0]))
print(all(abs(e) <= (2*index+1) for e in cat[1]) or all(abs(e) > (2*index+1) for e in cat[1]))

In [ ]:
# analysis of norm computation

means = []
for t in traces:
    means.append(np.mean(t[1529:1541]))
means = np.array(means)

kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(means.reshape(-1,1))
predictions = kmeans.predict(means.reshape(-1,1))
cat = {0:[], 1:[], 2:[]}
for i in range(10000):
    if x[i] < 128:
        cat[predictions[i]].append(x[i])
    else:
        cat[predictions[i]].append(x[i]-256)

print(all((e & 4) == 0 for e in cat[0]) and all(e >= 0 for e in cat[2]))
print(all(e < 0 for e in cat[1]))
print(all((e & 4) == 4 for e in cat[2]) and all(e > 0 for e in cat[2]))